# Manual for feature selection

## loading the data

You will load your data as an object of the MetabolitePhenotypeFeatureSelection class, so you can apply the funtions of this class to your data. Therefore, you will need to give the object a name to which you will refer when using these functions. In this example, I will call my object 'example'. An object of this class needs two datasets: one with the phenotypes per sample and one with the cleaned metabolic data. To specify the names of your data files and where to find them, you enter the path to your metabolic data at 'metabolome_csv=' and the phenotypic data at 'phenotype_csv=' as done in the example. Next, you specify the name of the column with the sample IDs in the phenotypic dataset at 'phenotype_sample_id='.

Make sure both your datasets are in a .csv format, uses '_' instead of ' ' (so no spaces), and is in lower case (so no capitals). 

For the metabolic data, the first column of your data should contain the feature IDs (so the identifier of the metabolic feature) and the other columns should be the samples. The values in the dataframe should be positive or zero. Next, you specify the name of the column with the feature IDs at 'metabolome_feature_id_col='. Ideally, this dataset is the output file from the write_clean_metabolome_to_csv() function from the MetaboliteAnalysis class. Therefore, the example data in this manual is the output from 'metabolite_data_filtering_manual.ipynb'. So the top of your dataset should look something like this:

| feature_id    | group1_1  | group1_2  | group1_3  | group1_4  | group2_1  | group2_2  | group2_3  | group2_4  | group3_1  | group3_2  | group3_3  | group3_4  | group4_1  | group4_2  | group4_3  | group4_4  |
|---------------|-----------|-----------|-----------|-----------|-----------|-----------|-----------|-----------|-----------|-----------|-----------|-----------|-----------|-----------|-----------|-----------|
| feature_1     | 0         | 0         | 0         | 0         | 1906      | 1586      | 1720      | 1216      | 0         | 132       | 0         | 0         | 3964      | 2620      | 2900      | 2304      |
| feature_2     | 1282      | 1094      | 5042      | 2140      | 174       | 0         | 0         | 0         | 0         | 412       | 746       | 660       | 0         | 302       | 720       | 986       |


For the phenotypic dataset, the first column should contain the sample IDs, which should be the same as the sample column names of the metabolic dataset. The second column should contain should contain the two phenotypes or classes which you have assigned to the samples. In the example data, the group1 and group2 samples have the phenotype 'resistant' and the group3 and group4 samples the phenotype 'susceptible'. So this dataset should look something like this:

| sample_id | phenotype   | 
|-----------|-------------|
| group1_1  | resistant   |
| group1_2  | resistant   |
| group1_3  | resistant   |
| group1_4  | resistant   |
| group2_1  | resistant   |
| group2_2  | resistant   |
| group2_3  | resistant   |
| group2_4  | resistant   |
| group3_1  | susceptible |
| group3_2  | susceptible |
| group3_3  | susceptible |
| group3_4  | susceptible |
| group4_1  | susceptible |
| group4_2  | susceptible |
| group4_3  | susceptible |
| group4_4  | susceptible |


First load the MetabolitePhenotypeFeatureSelection class. If your working with phloemfinder as a downloaded package, use:

In [ ]:
import numpy as np

from phloemfinder.feature_selection_using_ml import MetabolitePhenotypeFeatureSelection 

If your working with phloemfinder as a local github clone, you will need to open 'feature_selection_using_ml.py' in phloemfinder/src/phloemfinder. In this file change line 26 from

```
from phloemfinder.utils import compute_metrics_classification 
```
to
```
from utils import compute_metrics_classification 
```

After doing so, safe the changes, close and re-open your code editor and use:

In [1]:
import sys
sys.path.append('../src/phloemfinder/')

from feature_selection_using_ml import MetabolitePhenotypeFeatureSelection

/Users/lissydenkers/miniconda3/envs/sklearn/lib/python3.9/site-packages/tpot/builtins/__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")


Then load your data as object of the MetaboliteAnalysis class:

In [2]:
example = MetabolitePhenotypeFeatureSelection(
    metabolome_csv='./data_for_manuals/cleaned_metabolite_example_data.csv',    
    phenotype_csv='./data_for_manuals/phenotype_example_data.csv',
    metabolome_feature_id_col='feature_id', 
    phenotype_sample_id='sample_id')

After loading the data, we need to validate both datasets. 

First, we need to check if the metabolome input data doesn't contain any negative values:

In [3]:
example.validate_input_metabolome_df()

Metabolome data validated.


Next, we validate the phenotypic data. Specify the name of the column with the phenotypes (or classes) with 'phenotype_class_coll='. For the example data, this column is called "phenotype", as you can see is specified below. 

This validation step checks if the column you specified is present in your dataset and if you used a binary phenotype in this column. So, for the example data this means that we used "resistant" and "susceptible" as phenotype options, but this could also be "good" and "bad", or "red" and "blue". You could even use numbers, as long as the phenotype is binary. Personally, I prefer a descriptive phenotype, so that I will still understand what it means when coming back to it after a year.

In [4]:

example.validate_input_phenotype_df(phenotype_class_col='phenotype')

Phenotype data validated.


## Machine Learning

Now that we have loaded and validated all the data, it is time to start associating the phenotypes to the metabolic profiles. Because each dataset is different, the optimal Machine Learning pipeline is different for each dataset as well. 

### Baseline performance

Before we can build the optimal pipeline for our dataset, we need to set a baseline for the performance. We'll do this by running a simple Random Forest that should only take a few seconds. This baseline gives an idea of the performance we can expect from the more intricate pipeline we'll build in the next step. The performance of that pipeline should be better than the baseline performance.

The parameters for this function are:

* kfold: integer, default=5 <br> Cross-validation strategy to mitigate split effects on small datasets. <br> Default is to use a 5-fold cross-validation. <br> Has to be between 3 and 10. <br> For more information, see https://scikit-learn.org/stable/modules/cross_validation.html
        
* train_size: float or integer, default=0.8 <br> If float, should be between 0.5 and 1.0 and represent the proportion of the dataset to include in the train split. <br> If int, represents the absolute number of train samples. If None, the value is automatically set to the complement of the test size.

* random_state: integer, default=123 <br> Controls both the randomness of the train/test split samples used when building trees and the sampling of the features to consider when looking for the best split at each node.

* scoring_metric: str, default='balanced_accuracy' <br> A valid scoring value for the performance of the model. <br> 'balanced accuracy' is the average of recall obtained on each class. <br> To get a complete list, type: <br> >> from sklearn.metrics import SCORERS <br> >> sorted(SCORERS.keys())

In [5]:
example.get_baseline_performance(
    kfold=5, 
    train_size=0.8,
    random_state=123,
    scoring_metric='balanced_accuracy')

====== Training a basic Random Forest model =======
Average balanced_accuracy score on training data is: 65.000 % -/+ 37.40


====== Performance on test data of the basic Random Forest model =======
Average balanced_accuracy score on test data is: 100.000 %


### Feature selection

With the baseline set, you can build the pipeline for the selection of interesting features. This function is a wrapper for the Auto Machine Learning package TPOT. TPOT assembles the best fitting Machine Learning pipeline with many options for preprocessors and models using genetic programming. For this function, we have selected four models:

* DecisionTreeClassifier

* RandomForestClassifier

* GradientBoostingClassifier

* XGBClassifier

and 13 preprocessors: 

* Binarizer

* FeatureAgglomeration

* MaxAbsScaler

* MinMaxScaler

* Normalizer

* Nystroem

* PCA

* PolynomialFeatures

* RBFSampler

* RobustScaler

* StandardScaler

* ZeroCount

* OneHotEncoder

as options for the pipeline. <br> For more information on TPOT, see: https://epistasislab.github.io/tpot/.

The parameters for this function are:

* class_of_interest: string <br> The name of the class of interest (sometimes also called "positive class"). <br> This class will be used to calculate a recall and a precision score as follows: <br> $$ Recall = TP / (TP + FN) $$ <br> $$ Precision = TP / (TP + FP) $$ <br> where TP = true positives, FP = false positives, and FN = false negatives. 

* scoring_metric: string, default='balanced accuracy' <br> The function used to evaluate the quality of a given pipeline for the classification problem. <br> The following built-in scoring functions can be used:

  * 'accuracy'

  * 'adjusted_rand_score'

  * 'average_precision'

  * 'balanced_accuracy' 

  * 'f1'

  * 'f1_macro'

  * 'f1_micro'

  * 'f1_samples'

  * 'f1_weighted'

  * 'neg_log_loss' 

  * 'precision' etc. (suffixes apply as with ‘f1’)

  * 'recall' etc. (suffixes apply as with ‘f1’) 

  * ‘jaccard’ etc. (suffixes apply as with ‘f1’)

  * 'roc_auc'

  * ‘roc_auc_ovr’

  * ‘roc_auc_ovo’

  * ‘roc_auc_ovr_weighted’
  
  * ‘roc_auc_ovo_weighted’ 

* kfolds: integer, default=3 <br> Cross-validation strategy to mitigate split effects on small datasets. <br> Default is 3-fold cross-validation. <br> Has to be between 3 and 10. <br> For more information, see https://scikit-learn.org/stable/modules/cross_validation.html

* train_size: float or integer, default=0.8 <br> If float, should be between 0.5 and 1.0 and represent the proportion of the dataset to include in the train split. <br> If int, represents the absolute number of train samples. If None, the value is automatically set to the complement of the test size.

* max_time_mins: integer, default=5 <br> The time in minutes TPOT can use to optimize the pipeline (in total). This setting will allow TPOT to run until the specified time has elapsed and then stops the optimization process.

* max_eval_time_mins: float, default=1 <br> The time in minutes TPOT can use to evaluate a single pipeline. <br> This time has to be shorter than the 'max_time_mins' setting.

* random_state: integer, default=123 <br> Controls both the randomness of the train/test split samples used when building trees and the sampling of the features to consider when looking for the best split at each node.
    
* n_permutations: integer, default=10 <br> Number of permutations used to compute feature importances from the best model using the scikit-learn permutation_importance() method.

* export_best_pipeline: boolean, default=True <br> If True, the best fitting pipeline is exported as .py file. This allows for reuse of the pipeline on new datasets.

* path_for_saving_pipeline: string, default="./best_fitting_pipeline.py" <br> The path and filename of the best fitting pipeline to save. <br> The name must have a '.py' extension. 
  

When running this function with your real data, it would be good to do a quick test run (for example for the duration of a coffee break), to make sure everything is correct. If it looks fine, you can change the settings to overnight (depending on the time at which you start the run and at what time you want to use your computer again; for example max_time_mins=900). 

The 'Performance of ML model on train data' output ranges between 0% and 100%, while the 'Performance of ML model on test data' output ranges between 0 and 1. In all cases, higher means better.

If the model is performing much better on the train data than on the test data, it means the model is overfitted and therefore not robust. 

The most important value from the test data is the 'recall'. A high recall means a low false positive rate.

While running this step, try to close most other windows on your computer so it can work as efficiently as possible in the time you gave it. Oh, and make sure it doesn't go to sleep, otherwise it won't be doing anything.

For the example, we'll be running the function with the default settings:

In [6]:
example.search_best_model_with_tpot_and_compute_pc_importances(
    class_of_interest='resistant',
    scoring_metric='balanced_accuracy',
    kfolds=3,
    train_size=0.8,
    max_time_mins=5,
    max_eval_time_mins=1,
    random_state=123,
    n_permutations=10,
    export_best_pipeline=True,
    path_for_saving_pipeline="./best_fitting_pipeline.py")

Version 0.12.0 of tpot is outdated. Version 0.12.2 was released Friday February 23, 2024.


                                                                              
Generation 1 - Current best internal CV score: 0.75
                                                                              
Generation 2 - Current best internal CV score: 0.8333333333333334
                                                                              
Generation 3 - Current best internal CV score: 0.8333333333333334
                                                                              
Generation 4 - Current best internal CV score: 0.8333333333333334
                                                                              
5.29 minutes have elapsed. TPOT will close down.                              
TPOT closed during evaluation in one generation.
                                                                              
                                                                              
TPOT closed prematurely. Will use the current best pipeline.
        

After five minutes of training, we get a pipeline with a balanced accuracy score for the training data of 100% and for the test data of 0.75 (or 75%). This means that the pipeline performs better on the training data than on the test data and is thus overfitted on the training data. Although the precision is only 0.667 (meaning the false negative rate is quite high), the recall is 1, so there are no false positives. In this case, this means that this pipeline will only label a sample as 'resistant' (the class of interest) when it is indeed resistant, but will label some samples as 'susceptible' while they are actually resistant. A run of only five minutes is very short and allowing TPOT to run longer will improve the performance.

During the run, the function of 'Current best internal CV score' per generation, showing the cross-validation score of the best scoring pipeline per generation. After the run, a table is printed with the performance scores of the best fitting model/pipeline.

To have a look at the final pipeline, you can run:

In [7]:
example.best_model

Pipeline(steps=[('featureunion',
                 FeatureUnion(transformer_list=[('functiontransformer-1',
                                                 FunctionTransformer(func=<function copy at 0x1009360d0>)),
                                                ('functiontransformer-2',
                                                 FunctionTransformer(func=<function copy at 0x1009360d0>))])),
                ('standardscaler', StandardScaler()),
                ('randomforestclassifier',
                 RandomForestClassifier(bootstrap=False, max_features=0.05,
                                        min_samples_leaf=4, min_samples_split=7,
                                        n_estimators=1000, random_state=123))])

### Extracting the results
To deal with the strong correlation between the features that is characteristic for metabolic data, the data is first 'flattened' into principal components (PCs) by the search_best_model_with_tpot_and_compute_pc_importances() function. These PCs are then used as features in the pipeline. 

The first step to extract the interesting features from the created pipeline, is to get the important PCs. The PC that is the most important for the model, gets the highest variable importance (var_imp). Sometimes, there are only a few PCs with a mean_var_imp>0. Other times there are many, meaning you will have to decide on a threshold for which are interesting enough.

You can print the variable importances with:

In [8]:
print(example.pc_importances)

      mean_var_imp  std_var_imp     perm0     perm1  perm2  perm3  perm4  \
pc                                                                         
PC0       0.033333     0.040825  0.083333  0.083333    0.0    0.0    0.0   
PC1       0.033333     0.040825  0.083333  0.083333    0.0    0.0    0.0   
PC4       0.033333     0.040825  0.083333  0.083333    0.0    0.0    0.0   
PC10      0.025000     0.038188  0.000000  0.083333    0.0    0.0    0.0   
PC2       0.000000     0.000000  0.000000  0.000000    0.0    0.0    0.0   
PC3       0.000000     0.000000  0.000000  0.000000    0.0    0.0    0.0   
PC5       0.000000     0.000000  0.000000  0.000000    0.0    0.0    0.0   
PC6       0.000000     0.000000  0.000000  0.000000    0.0    0.0    0.0   
PC7       0.000000     0.000000  0.000000  0.000000    0.0    0.0    0.0   
PC8       0.000000     0.000000  0.000000  0.000000    0.0    0.0    0.0   
PC9       0.000000     0.000000  0.000000  0.000000    0.0    0.0    0.0   
PC11      0.

For the test data, you can see that PC0, PC1 and PC4 all have a mean variable importance of 0.033 and PC10 has a mean variable importance of 0.025.

Once you know which PCs are interesting, you can extract the features most important for these PCs.

**!!You have to be careful with the PC number here!!** In the list of PC importances above, the PC numbers start at 0. In this function, however, the PC numbers are +1 to prevent some errors. So if you are interested in PC3 from the list above, you should specify 'selected_pc=4' in this function.

* selected_pc: integer, default=1 <br> The Principal Component of which you want to know the most important features. 
* top_n: integer, default=5 <br> Number of features to select. The top_n features with the highest absolute loadings will be selected from the selected_pc you specified. 

For the example data we'll take a look at the top 10 most important features of PC0 from the list (so we specify selected_pc=1 for this next function). Next to the names of the features, this function will also produce the loading scores. The higher this score, the more important the feature is for the selected PC.

In [9]:
example.get_names_of_top_n_features_from_selected_pc(
    selected_pc=1,
    top_n=10)

Here are the metabolite names with the top 10 absolute loadings on PC1


,feature_name,loading
0,feature_8913,0.355209
1,feature_9321,0.311647
2,feature_5216,0.299312
3,feature_8850,0.270446
4,feature_5162,0.196034
5,feature_3928,0.179552
6,feature_9803,0.176027
7,feature_10785,0.175505
8,feature_8953,0.172160
9,feature_2802,0.167416


After extracting the top features from all interesting PCs, you can go back to your data to for example have a look at the abundance of the features per group, or annotate them.